# Benchmarking on popular simulators
This notebook reproduces the results presented in Table 1 of the paper.

In [1]:
import numpy as np
import scipy.stats as stats # used for inverse Gaussian
import scipy.spatial.distance as distance # distance used for kernel
import matplotlib.pyplot as plt
import probnum as pn
import random

from probnum.quad._integration_measures import (
    GaussianMeasure,
    IntegrationMeasure,
    LebesgueMeasure,
)
from probnum.randprocs.kernels import ExpQuad, Kernel, Matern, ProductMatern

## Functions

In [2]:
#Function generating standard normals using the inverse of the univariate Gaussian CDF:
def normals_inv(u):
  # avoid origin
  u[u==0] = np.nextafter(0, 1)
  # create standard normal samples
  z = stats.norm.ppf(u, loc=0, scale=1)
  return z

def boxmuller(unif1,unif2):
  u1 = np.sqrt(-2*np.log(unif1))*np.cos(2*np.pi*unif2)
  u2 = np.sqrt(-2*np.log(unif1))*np.sin(2*np.pi*unif2)
  return np.transpose(np.vstack([u1,u2]))

def normals(n, d, unif, sv=False):

    # avoid origin
    unif[unif==0] = np.nextafter(0, 1)

    # if d is odd, add one dimension
    if d % 2 != 0:
      dim = d + 1
    else:
      dim = d

    # expand dimensions for SV model
    if sv == True:
      dim = 2+2*d

    # create standard normal samples
    u = np.zeros((n,dim))
    for i in np.arange(0,dim,2):
      u[:,i:(i+2)] = boxmuller(unif[:,i],unif[:,(i+1)])

    # if d is odd, drop one dimension
    if d % 2 != 0 or sv == True:
      u = np.delete(u,-1,1)

    return u

# Generator for univariate g-and-k distribution
def sample_gandk(n, theta, method = "MC"):
    
    a = theta[0]
    b = theta[1]
    g = theta[2]
    k = theta[3]
    
    if method == "MC":
        z_unif = np.random.rand(n,1)
    elif method == "QMC":
        sampler = stats.qmc.Sobol(d=1, scramble=True)
        z_unif = sampler.random(n)
#         z_unif = sampler.random_base2(m=8)
        
    z = normals_inv(z_unif)
    x = np.zeros(shape = (n,1))
    
    for i in range(n):
        x[i] = a+b*(1+0.8*((1-np.exp(-g*z[i]))/(1+np.exp(-g*z[i]))))*((1+z[i]**2)**(k))*z[i]
    return z_unif,z,x

# Function estimate the MMD-squared using the V-statistic
def MMD_unweighted(x,y, lengthscale):
    """ Approximates the squared MMD between samples x_i ~ P and y_i ~ Q
    """
    
    if len(x.shape) ==1:
        x = np.array(x, ndmin = 2).transpose()
        y = np.array(y, ndmin = 2).transpose()
        
    m = x.shape[0]
    n = y.shape[0]
    
    z = np.concatenate((x,y), axis = 0)
    
    K = kernel_matrix(z,z, lengthscale)
    
    kxx = K[0:m, 0:m]
    kyy = K[m:(m+n), m:(m+n)]
    kxy = K[0:m, m:(m+n)]
    
    return (1/m**2) * np.sum(kxx) - (2/(m*n)) * np.sum(kxy) + (1/n**2) * np.sum(kyy)

# Function to set the lengthscale of the kernel using median heuristic
def median_heuristic(y):
    a = distance.cdist(y, y,'sqeuclidean')
    return np.sqrt(np.median(a / 2))

# Function to compute the kernel Gram matrix
def kernel_matrix(x,y, l):
    if len(x.shape) ==1:
        x = np.array(x, ndmin = 2).transpose()
        y = np.array(y, ndmin = 2).transpose()
        
    return np.exp(-(1/(2*l**2)) * distance.cdist(x,y,'sqeuclidean'))

# Function to compute the kernel mean embedding wrt Lebesgue measure
def embedding_unif(u):
    # Function to compute the embedding when U is uniformly distributed
   
    # Compute lengthscale for kernel c    
    l = median_heuristic(u)

    dim = u.shape[1]
    z = np.zeros(shape = u.shape)
    
    for i in range(dim):
        z[:,i] = np.sqrt(2*np.pi) * l * (stats.norm.cdf(1, loc = u[:,i], scale = l) -
                                      stats.norm.cdf(0, loc = u[:,i], scale = l))
    if dim == 1:
        return z
    else:
        return np.prod(z, axis = 1)
    
# Function estimate the MMD-squared using our optimally-weighted (OW) estimator
def MMD_weighted(x, y, w, lengthscale):
#     """ Optimally weighted squared MMD estimate between samples x_i ~ P and y_i ~ Q
#     """
    
    if len(x.shape) ==1:
        x = np.array(x, ndmin = 2).transpose()
        y = np.array(y, ndmin = 2).transpose()
        w = np.array(w, ndmin = 2).transpose()
        
    m = x.shape[0]
    n = y.shape[0]

    xy = np.concatenate((x,y), axis = 0)

    K = kernel_matrix(xy, xy, lengthscale)

    kxx = K[0:m, 0:m]
    kyy = K[m:(m+n), m:(m+n)]
    kxy = K[0:m, m:(m+n)]

        # first sum
    sum1 = np.matmul(np.matmul(w.transpose(), kxx), w)

        # second sum
    sum2 = np.sum(np.matmul(w.transpose(), kxy))

        # third sum
    sum3 = (1/n**2) * np.sum(kyy)

    return sum1 - (2/(n)) * sum2 + sum3

# Function to compute the optimal weights given random variables u_1, ..., u_m ~ U
def computeWeights(u, z):
    m = u.shape[0]
    
    # Compute lengthscale for kernel c    
    l = median_heuristic(u)

    # Compute Gram-matrix C
    delta = 1e-8
    C = kernel_matrix(u, u, l) + delta * np.identity(m)

    C_inv = np.linalg.inv(C)
    
    return np.matmul(C_inv, z)

### Univariate g-and-k distribution

In [3]:
# Parameter settings
m = 256                                   # number of simulated samples
n = 10000                                 # number of true samples
theta = np.array([3,1,0.1,0.1])           # true theta
d = 1                                     # dimensions of data
p = len(theta)                            # dimensions of parameter space

In [57]:
N = 100 # No. of times the experiment is repeated

MMDvals_Vstat = np.zeros(shape = (N,1))
MMDvals_weighted = np.zeros(shape = (N,1))

for i in range(N):
    random.seed(i)
    # generate samples from g-and-k distribution
    _,_,y = sample_gandk(n,theta, method = "MC") # Change method to "MC" for Monte Carlo samples and "QMC" for RQMC points
    u,u_norm,x = sample_gandk(m, theta, method = "MC")

    # Compute the lengthscale
    lengthscale = median_heuristic(y)

    MMDvals_Vstat[i] = MMD_unweighted(x, y, lengthscale)

    # Compute the embedding z (for uniform U)
    z = embedding_unif(u)

    # Compute optimal weights
    w = computeWeights(u, z)

    # Compute optimally-weighted MMD
    MMDvals_weighted[i] = MMD_weighted(x, y, w, lengthscale)

print(np.mean(MMDvals_Vstat) * 10**(3))
print(np.std(MMDvals_Vstat)* 10**(3))
print(np.mean(MMDvals_weighted)* 10**(3))
print(np.std(MMDvals_weighted)* 10**(3))

2.1521431821442967
1.4690552004295188
0.4585007503653893
0.2441935704453178


In [4]:
_,_,y = sample_gandk(n,theta, method = "MC") # Change method to "MC" for Monte Carlo samples
u,u_norm,x = sample_gandk(m, theta, method = "QMC")

# Compute the lengthscale
lengthscale = median_heuristic(y)

print(MMD_unweighted(x, y, lengthscale)* 10**(3))

# Compute the embedding z (for uniform U)
z = embedding_unif(u)

    # Compute optimal weights
w = computeWeights(u, z)
print(MMD_weighted(x, y, w, lengthscale)* 10**(3))

0.03704023208805918
[[0.03737484]]


### Two moons model

In [5]:
def twoMoons(theta, n, method = "MC", exponent = 8):
    # Implementation from Wiqvist 2021
    
    if method == "MC":
        u = np.random.rand(n, 2)
    elif method == "QMC":
        sampler = stats.qmc.Sobol(d=2, scramble=True)
        u = sampler.random_base2(exponent)
        
    z = normals_inv(u[:,1])

    a = -np.pi/2 + np.pi * u[:,0]
    r = 1.0 + 0.1 * z

    p = np.zeros(shape = (n,2))
    p[:,0] = np.cos(a) + 1
    p[:,1] = r * np.sin(a)

    x = p + np.array([- np.abs(theta[0] + theta[1]) / np.sqrt(2), (-theta[0] + theta[1]) / np.sqrt(2)])
    
    return u,x

In [23]:
m = 256               # number of simulated samples
n = 10000             # number of observed samples
theta = [0.5, 0.5]    # true theta

N = 100               # number of repetitions

MMDvals_Vstat = np.zeros(shape = (N,1))
MMDvals_weighted = np.zeros(shape = (N,1))

for i in range(N):
#     random.seed(i)
    # generate samples from two moons model
    _,y = twoMoons(theta, n, method = "MC", exponent = 8) # Change "MC" to "QMC" to generate RQMC points instead of iid
    u,x = twoMoons(theta, m, method = "MC", exponent = 8)

    # Compute the lengthscale
    lengthscale = median_heuristic(y)

    # Compute MMD using V-statistic
    MMDvals_Vstat[i] = MMD_unweighted(x, y, lengthscale)

    # Compute the embedding z (for uniform U)
    z = embedding_unif(u)

    # Compute optimal weights
    w = computeWeights(u, z)

    # Compute optimally-weighted MMD
    MMDvals_weighted[i] = MMD_weighted(x, y, w, lengthscale)
    
print(np.mean(MMDvals_Vstat)*10**(3))
print(np.std(MMDvals_Vstat)*10**(3))
print(np.mean(MMDvals_weighted) * 10**(3))
print(np.std(MMDvals_weighted)*10**(3))

### Moving Average (MA2) model

In [84]:
def moving_average_2(θ1, θ2, d, n, method = "MC"):
    
    if method == "MC":
        u_unif = np.random.rand(n, d+2)
    elif method == "QMC":
        sampler = stats.qmc.Sobol(d= d+2, scramble=True)
        u_unif = sampler.random_base2(8)
#         u_unif = sampler.random(n)

    y = np.zeros(shape = (n,d))
    
    for i in range(n):
        λ = normals_inv(u_unif[i,:])
        y[i,:] = λ[2:] + θ1*λ[1:-1] + θ2*λ[:-2]
        
    return u_unif,y

d = 10            # Dimension of data
n = 10000         # number of observed samples
m = 256           # number of simulated samples
theta1 = 0.6
theta2 = 0.2

In [87]:
N = 100

MMDvals_Vstat = np.zeros(shape = (N,1))
MMDvals_weighted = np.zeros(shape = (N,1))

for i in range(N):
    # generate samples from ma(2) model
    _,y = moving_average_2(theta1, theta2, d, n, method = "MC") # Change "MC" to "QMC" to generate RQMC points instead of iid
    u,x = moving_average_2(theta1, theta2, d, m, method = "MC")

    # Compute the lengthscale
    lengthscale = median_heuristic(y)

    # Compute MMD using V-statistic
    MMDvals_Vstat[i] = MMD_unweighted(x, y, lengthscale)

    # Compute the embedding z (for uniform U)
    z = embedding_unif(u)

    # Compute optimal weights
    w = computeWeights(u, z)

    # Compute optimally-weighted MMD
    MMDvals_weighted[i] = MMD_weighted(x, y, w, lengthscale)
    
print(np.mean(MMDvals_Vstat[~np.isnan(MMDvals_Vstat)]))
print(np.std(MMDvals_Vstat[~np.isnan(MMDvals_Vstat)]))
print(np.mean(MMDvals_weighted[~np.isnan(MMDvals_weighted)]))
print(np.std(MMDvals_weighted[~np.isnan(MMDvals_weighted)]))    

### Bivariate Beta distribution

In [89]:
# Function to sample MC or QMC points from bivariate beta distribution 
def sample_bibeta(method_sampling,n,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  # split theta into integer and decimal parts
  i_theta,_ = divmod(theta,np.ones(5)) 
  p = np.sum(i_theta.astype(int))

  # sample uniforms
  if method_sampling == 'MC':
    unif = np.random.rand(n,p)
  if method_sampling == 'QMC':
    sampler = stats.qmc.Sobol(d = p, scramble=True)
    unif = sampler.random_base2(8)

  # generate samples
  if method_sampling == 'QMC' or method_sampling == 'RQMC':
    x = gen_bibeta(theta,unif,qmc_gamma=True)
  else: 
    x = gen_bibeta(theta,unif)

  return unif, x

# Simulator for bivariate beta distribution
def gen_bibeta(theta, unif, qmc_gamma=False):
  # number of samples
  n = unif.shape[0]

  # split theta into integer and decimal parts
  i_theta,d_theta = divmod(theta,np.ones(5)) 
  i_theta = i_theta.astype(int)

  # initialise
  utild = np.zeros([n,5])
  x = np.zeros((n,2))

  # logs on uniforms
  logunif = np.log(unif)

  # get \tilde{u}
  j=0
  for i in range(5):
    sum = np.zeros(n)
    if i_theta[i]!=0:
      for k in range(i_theta[i]):
        sum[:] += logunif[:,k+j]
      utild[:,i] = -sum
    if d_theta[i]!=0:
      if qmc_gamma:
        utild[:,i] += sample_qmc_gamma(d_theta[i],n)
      else:
        utild[:,i] += np.random.gamma(d_theta[i],1,n)
    j += i_theta[i]

  # generator
  x1 = np.sum(np.vstack([utild[:,0],utild[:,2]]),axis=0)/np.sum(np.vstack([utild[:,0],utild[:,2],utild[:,3],utild[:,4]]),axis=0)
  x2 = np.sum(np.vstack([utild[:,1],utild[:,3]]),axis=0)/np.sum(np.vstack([utild[:,1],utild[:,2],utild[:,3],utild[:,4]]),axis=0)

  return np.vstack([x1,x2]).T

In [93]:
n = 10000                 # number of observed samples
m = 256                   # number of simulated samples
theta1 = (1,1,1,1,1)      # True theta

N = 100 # No. of times the experiment is repeated

MMDvals_Vstat = np.zeros(shape = (N,1))
MMDvals_weighted = np.zeros(shape = (N,1))

for i in range(N):
    # generate samples from bivariate beta distribution
    _, y = sample_bibeta('MC',n,theta1)

    u, x = sample_bibeta('MC',m,theta1)

    # Compute the lengthscale
    lengthscale = median_heuristic(y)

    MMDvals_Vstat[i] = MMD_unweighted(x, y, lengthscale)

    # Compute the embedding z (for uniform U)
    z = embedding_unif(u)

    # Compute optimal weights
    w = computeWeights(u, z)

    # Compute optimally-weighted MMD
    MMDvals_weighted[i] = MMD_weighted(x, y, w, lengthscale)

print(np.mean(MMDvals_Vstat)* 10**(3))
print(np.std(MMDvals_Vstat)* 10**(3))
print(np.mean(MMDvals_weighted)* 10**(3))
print(np.std(MMDvals_weighted)* 10**(3))

### MG1 Queuing model

In [6]:
def exp_inverse(u, l):
    x = -1 / l * np.log(1 - u)
    return x

def MG1queue(theta, n, nTime, method = "MC"):
    theta1 = theta[0]
    theta2 = theta[1] + theta[0]
    theta3 = theta[2]
    
    if method == "MC":
        u1 = np.random.rand(n, nTime*2)
    elif method == "QMC":
        sampler = stats.qmc.Sobol(d = nTime*2, scramble=True)
        u1 = sampler.random_base2(8)
    
    data = np.zeros(shape = (n, nTime))
    
    for i in range(n):
        u = theta1 + (theta2 - theta1) * u1[i, 0:nTime]
        v = np.zeros(nTime)
        y = np.zeros(nTime)
        x = np.zeros(nTime)
        e = exp_inverse(u1[i, nTime:], theta3)
        v[0] = e[0]
        y[0] = u[0] + v[1]
        x[1] = y[1]
        
        for t in range(1,nTime):
            v[t] = v[t-1] + e[t]
            y[t] = u[t] + np.max((0, v[t] - x[t-1]))
            x[t] = x[t-1] + y[t]
        
        data[i, :] = y
        
    return u1, data

In [17]:
n = 10000 # number of observed samples
m = 500   # number of simulated samples
nTime = 5 # dimension of data

theta = [1, 5, 0.2] # True theta

In [132]:
N = 100   # No. of times the experiment is repeated

MMDvals_Vstat = np.zeros(shape = (N,1))
MMDvals_weighted = np.zeros(shape = (N,1))

for i in range(N):

    _, y = MG1queue(theta, n, nTime)
    u, x = MG1queue(theta, m, nTime, method = "MC") # Change "MC" to "QMC" to generate RQMC points instead of iid

    # Compute the lengthscale
    lengthscale = median_heuristic(y)

    MMDvals_Vstat[i] = MMD_unweighted(x, y, lengthscale)

    z = embedding_unif(u)

    # Compute optimal weights
    w = computeWeights(u, z)

    # Compute optimally-weighted MMD
    MMDvals_weighted[i] = MMD_weighted(x, y, w, lengthscale)
    
print(np.mean(MMDvals_Vstat)* (10**3)) 
print(np.std(MMDvals_Vstat)* (10**3))
print(np.mean(MMDvals_weighted)* (10**3))
print(np.std(MMDvals_weighted)* (10**3))

### Lotka-Volterra model
The file size of the data used for this experiment is too large to include with the submission. Hence, the following experiment is commented out.

In [211]:
# # IID points
# uvals = np.load("uvals.npy")
# xvals = np.load("xvals.npy")

# n = 10000 # number of observed samples
# m = 256   # number of simulated samples

# y = xvals[0:n, :]
# lengthscale = median_heuristic(y)

In [96]:
# N = 100   # No. of times the experiment is repeated

# MMDvals_Vstat = np.zeros(shape = (N,1))
# MMDvals_weighted = np.zeros(shape = (N,1))

# for i in range(N):

#     x = xvals[(n + i*m):(n + i*m + m), :]

#     u = uvals[(n + i*m):(n + i*m + m), :]

#     # Compute the lengthscale
#     lengthscale = median_heuristic(y)

#     MMDvals_Vstat[i] = MMD_unweighted(x, y, lengthscale)

#     z = embedding_unif(u)

#     # Compute optimal weights
#     w = computeWeights(u, z)

#     # Compute optimally-weighted MMD
#     MMDvals_weighted[i] = MMD_weighted(x, y, w, lengthscale)
#     print(i)
 
# print(np.mean(MMDvals_Vstat)* 10**(3))
# print(np.std(MMDvals_Vstat)* 10**(3))
# print(np.mean(MMDvals_weighted)* 10**(3))
# print(np.std(MMDvals_weighted)* 10**(3))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [212]:
# # QMC points
# uvals_qmc = np.load("uvals_QMC.npy")
# xvals_qmc = np.load("xvals_QMC.npy")
# m = 256

# N = 100   # No. of times the experiment is repeated

# MMDvals_Vstat = np.zeros(shape = (N,1))
# MMDvals_weighted = np.zeros(shape = (N,1))

# for i in range(N):

#     x = xvals_qmc[0:m, :, i]

#     u = uvals_qmc[0:m, :, i]

#     # Compute the lengthscale
#     lengthscale = median_heuristic(y)

#     MMDvals_Vstat[i] = MMD_unweighted(x, y, lengthscale)

#     z = embedding_unif(u)

#     # Compute optimal weights
#     w = computeWeights(u, z)

#     # Compute optimally-weighted MMD
#     MMDvals_weighted[i] = MMD_weighted(x, y, w, lengthscale)
#     print(i)

# print(np.mean(MMDvals_Vstat)* 10**(3))
# print(np.std(MMDvals_Vstat)* 10**(3))
# print(np.mean(MMDvals_weighted)* 10**(3))
# print(np.std(MMDvals_weighted)* 10**(3))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
